In [118]:
import re
import mechanize
import xlsxwriter
from lxml import html
import os
import csv
from collections import defaultdict
import pandas as pd
from pathlib import Path # For data paths
import numpy as np
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

pd.set_option('display.max_rows', 500)

In [119]:
def tableDataText(table):    
    """Parses a html segment started with tag <table> followed 
    by multiple <tr> (table rows) and inner <td> (table data) tags. 
    It returns a list of rows with inner columns. 
    Accepts only one <th> (table header/data) in the first row.
    """
    def rowgetDataText(tr, coltag='td'): # td (data) or th (header)       
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]  
    rows = []
    trs = table.find_all('tr')
    headerow = rowgetDataText(trs[0], 'th')
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append(rowgetDataText(tr, 'td') ) # data row       
    return rows



In [149]:
def format_color_groups(df):
    colors = ['gold', 'lightblue']
    x = df.copy()
    factor = 0.5

    style = f'background-color: {colors[0]}'
    style2 = f'background-color: {colors[0]}'
    x.loc[x['Difference'] < factor, :] = style
        
    return x

In [120]:
headers = {'Accept-Encoding': 'identity'}
try:
    response = requests.get("https://www.borzamalta.com.mt/?handler=TradingBoard", headers=headers)
except requests.ConnectionError as error:
    print(error)

In [123]:
list_table=[]
bs = BeautifulSoup(response.content)
tables = bs.findAll('table')
for table in tables:
    df = pd.DataFrame(tableDataText(table))
    df=df.rename(columns=df.iloc[0]).drop(df.index[0])
    list_table.append(df)

Table 2 - Corporate Bonds
Table 5 - Regulated Market
Table 7 - Corporate Bonds offers

In [129]:
trades = list_table[5].set_index('Symbol Code').filter(regex='A$|B$', axis=0)
trades

,Traded Time,Currency,Volume Traded,Value Traded,Execution Price
Symbol Code,,,,,
SM31A,15:30:23:033,EUR,"5,000","5,090.00",101.8000
SM31A,15:30:23:033,EUR,"5,000","5,090.00",101.8000
G41A,15:30:10:027,EUR,"10,000","9,200.00",92.0000
PC26A,15:03:56:097,EUR,"5,600","5,588.80",99.8000
PC26A,15:03:41:020,EUR,500,495.00,99.0000
PC26A,15:03:41:020,EUR,"1,500","1,497.00",99.8000
GO31A,14:51:36:053,EUR,"1,300","1,267.50",97.5000
G41A,14:46:46:030,EUR,"2,000","1,840.00",92.0000
IH26B,14:37:09:010,EUR,"15,000","15,000.00",100.0000


In [131]:
offers = list_table[7].set_index('Symbol Code').sort_values(by=['Best Offer Price'], ascending=False)
offers

,Security Name,Best Bid Count,Best Bid Volume,Best Bid Price,Best Offer Price,Best Offer Volume,Best Offer Count
Symbol Code,,,,,,,
MB29A,4% MeDirect Bank Malta plc 2029,-,-,-,99.980,"4,000",1
LS31A,4% LifeStar Insurance plc 2031,-,-,-,99.900,"30,000",1
HF28A,3.85% Hili Finance Company plc 2028,1,"5,000",97.810,99.800,300,1
HF29A,3.8% Hili Finance Company plc 2029,1,"10,000",97.300,99.800,"78,800",3
SH26A,4% Shoreline Mall plc 2026,2,"40,000",98.500,99.800,"5,000",1
PC26A,3.75% Premier Capital plc 2026,1,"7,400",98.800,99.800,"2,900",1
EN29A,4.5% Endo Finance plc 2029,-,-,-,99.500,"50,000",1
SH32A,4.5% Shoreline Mall plc 2032,1,"10,000",93.010,99.500,"10,000",1
BV31A,3.75% Bank Of Valletta plc 2031,-,-,-,99.490,"50,000",2


In [134]:
total_trades = list_table[2].set_index('Symbol Code')
total_trades

,Volume Traded,Value Traded,Trades,High Price,Low Price,Open Price,Closing Price,Change
Symbol Code,,,,,,,,
1923A,"5,600","5,740.00",2,102.500,102.500,102.500,102.500,0.000
6PM25,"8,000","8,160.00",1,102.000,102.000,102.000,102.000,1.000
AX24A,"4,000","4,160.00",1,104.000,104.000,104.000,104.000,2.000
AX32A,"18,000","17,924.70",5,100.000,99.400,99.400,100.000,0.600
BD24A,"10,000","10,250.00",1,102.500,102.500,102.500,102.500,0.000
BR31A,"4,000","4,040.00",1,101.000,101.000,101.000,101.000,0.500
DF26A,"22,000","22,176.00",3,100.800,100.800,100.800,100.800,0.800
G332A,"63,200","64,571.50",3,102.550,102.000,102.550,102.000,-1.000
GO31A,"15,000","14,697.00",5,98.500,97.500,98.000,97.500,-1.000


In [136]:
df = pd.merge(trades,offers[['Best Offer Price','Best Offer Volume']], left_index=True, right_index=True, how='left')

In [138]:
df2 = pd.merge(df, total_trades[['Closing Price']], left_index=True, right_index=True, how='left')

In [139]:
df2

,Traded Time,Currency,Volume Traded,Value Traded,Execution Price,Best Offer Price,Best Offer Volume,Closing Price
Symbol Code,,,,,,,,
1923A,09:45:41:030,EUR,"1,000","1,025.00",102.5000,102.500,"42,400",102.500
1923A,09:45:41:030,EUR,"4,600","4,715.00",102.5000,102.500,"42,400",102.500
AX24A,14:20:33:063,EUR,"4,000","4,160.00",104.0000,104.500,"13,300",104.000
AX32A,12:23:06:017,EUR,"3,000","2,999.70",99.9900,100.000,"6,600",100.000
AX32A,12:23:06:017,EUR,"6,600","6,567.00",99.5000,100.000,"6,600",100.000
AX32A,12:23:06:017,EUR,400,400.00,100.0000,100.000,"6,600",100.000
AX32A,11:27:59:067,EUR,"2,000","1,988.00",99.4000,100.000,"6,600",100.000
AX32A,11:27:59:067,EUR,"6,000","5,970.00",99.5000,100.000,"6,600",100.000
BD24A,11:34:35:080,EUR,"10,000","10,250.00",102.5000,-,-,102.500


In [140]:
offers_price = pd.merge(offers, total_trades[['Closing Price']], left_index=True, right_index=True, how='left')
offers_price

,Security Name,Best Bid Count,Best Bid Volume,Best Bid Price,Best Offer Price,Best Offer Volume,Best Offer Count,Closing Price
Symbol Code,,,,,,,,
MB29A,4% MeDirect Bank Malta plc 2029,-,-,-,99.980,"4,000",1,99.980
LS31A,4% LifeStar Insurance plc 2031,-,-,-,99.900,"30,000",1,NaN
HF28A,3.85% Hili Finance Company plc 2028,1,"5,000",97.810,99.800,300,1,NaN
HF29A,3.8% Hili Finance Company plc 2029,1,"10,000",97.300,99.800,"78,800",3,99.800
SH26A,4% Shoreline Mall plc 2026,2,"40,000",98.500,99.800,"5,000",1,NaN
PC26A,3.75% Premier Capital plc 2026,1,"7,400",98.800,99.800,"2,900",1,99.800
EN29A,4.5% Endo Finance plc 2029,-,-,-,99.500,"50,000",1,NaN
SH32A,4.5% Shoreline Mall plc 2032,1,"10,000",93.010,99.500,"10,000",1,NaN
BV31A,3.75% Bank Of Valletta plc 2031,-,-,-,99.490,"50,000",2,NaN


In [166]:
offers_price = offers_price.replace('-', np.nan)
offers_price['Difference'] = offers_price['Best Offer Price'].astype(float)-offers_price['Closing Price'].astype(float)
offers_price['Difference'] = offers_price['Difference'].round(2)
offers_price

,Security Name,Best Bid Count,Best Bid Volume,Best Bid Price,Best Offer Price,Best Offer Volume,Best Offer Count,Closing Price,Comment,Difference
Symbol Code,,,,,,,,,,
MB29A,4% MeDirect Bank Malta plc 2029,NaN,NaN,NaN,99.980,"4,000",1,99.980,yes,0.00
LS31A,4% LifeStar Insurance plc 2031,NaN,NaN,NaN,99.900,"30,000",1,NaN,no,NaN
HF28A,3.85% Hili Finance Company plc 2028,1,"5,000",97.810,99.800,300,1,NaN,no,NaN
HF29A,3.8% Hili Finance Company plc 2029,1,"10,000",97.300,99.800,"78,800",3,99.800,yes,0.00
SH26A,4% Shoreline Mall plc 2026,2,"40,000",98.500,99.800,"5,000",1,NaN,no,NaN
PC26A,3.75% Premier Capital plc 2026,1,"7,400",98.800,99.800,"2,900",1,99.800,yes,0.00
EN29A,4.5% Endo Finance plc 2029,NaN,NaN,NaN,99.500,"50,000",1,NaN,no,NaN
SH32A,4.5% Shoreline Mall plc 2032,1,"10,000",93.010,99.500,"10,000",1,NaN,no,NaN
BV31A,3.75% Bank Of Valletta plc 2031,NaN,NaN,NaN,99.490,"50,000",2,NaN,no,NaN


In [172]:
def is_offer_valid(temp):
    if  temp < 0.5:
        return 'background-color: green'
    if  temp > 1:
        return 'background-color: red'

s = offers_price.style.applymap(is_offer_valid, subset=['Difference'])
s

,Security Name,Best Bid Count,Best Bid Volume,Best Bid Price,Best Offer Price,Best Offer Volume,Best Offer Count,Closing Price,Comment,Difference
Symbol Code,,,,,,,,,,
MB29A,4% MeDirect Bank Malta plc 2029,nan,nan,nan,99.980,"4,000",1,99.980,yes,0.000000
LS31A,4% LifeStar Insurance plc 2031,nan,nan,nan,99.900,"30,000",1,nan,no,nan
HF28A,3.85% Hili Finance Company plc 2028,1,"5,000",97.810,99.800,300,1,nan,no,nan
HF29A,3.8% Hili Finance Company plc 2029,1,"10,000",97.300,99.800,"78,800",3,99.800,yes,0.000000
SH26A,4% Shoreline Mall plc 2026,2,"40,000",98.500,99.800,"5,000",1,nan,no,nan
PC26A,3.75% Premier Capital plc 2026,1,"7,400",98.800,99.800,"2,900",1,99.800,yes,0.000000
EN29A,4.5% Endo Finance plc 2029,nan,nan,nan,99.500,"50,000",1,nan,no,nan
SH32A,4.5% Shoreline Mall plc 2032,1,"10,000",93.010,99.500,"10,000",1,nan,no,nan
BV31A,3.75% Bank Of Valletta plc 2031,nan,nan,nan,99.490,"50,000",2,nan,no,nan
